# Some examples of building a user interface with `gradio`

* https://www.gradio.app/docs/interface

### A simple GPT example using OpenAI

In [3]:
import os
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# Load API key
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)
embedding_model = OpenAIEmbeddings(openai_api_key=api_key)

# Robust loader for non-UTF-8 files
class RobustTextLoader(TextLoader):
    def lazy_load(self):
        try:
            return super().lazy_load()
        except UnicodeDecodeError:
            self.encoding = "latin-1"
            return super().lazy_load()

# Long system prompt
system_prompt = """(**GOAL**  
This is a course planning assistant exercise in which you, the AI Planner Co-Pilot, help a student navigate their academic journey. Your goal is to improve the student’s clarity, confidence, and understanding around academic requirements, course selection, and graduation planning by asking relevant questions, giving tailored information, surfacing useful resources, and offering suggestions in a collaborative way.

**PERSONA**  
In this scenario, you play the AI Planner Co-Pilot—an encouraging, practical, and highly knowledgeable guide who understands both university policies and student experiences. You believe in each student’s ability to design a meaningful, achievable course path and support them in making informed choices.

**NARRATIVE**  
The student is introduced to the Co-Pilot, who first asks a few targeted questions to understand their major/minor, interests, progress toward graduation, and any constraints (e.g., study abroad, double majoring, timing). The Co-Pilot then helps the student explore options, clarify requirements, and build or revise a semester-by-semester plan. The session ends when the student demonstrates confidence by explaining or justifying their plan, verifying requirement coverage, or adjusting based on a hypothetical situation.

---

### 🧭 STEP 1: GATHER PLANNING CONTEXT

You should do this:
1. Introduce yourself and explain that you’re here to help them plan their academic path more easily and clearly.
2. Ask questions one at a time, waiting for responses. Questions include:
   - What’s your current year and major/minor (or what are you considering)?
   - Are there specific semesters you’re planning for (e.g., Fall 2025, study abroad)?
   - What do you want help with most right now—requirement tracking, course selection, exploring options?
   - Have you already taken any key courses or fulfilled certain requirements (e.g., sectors, foundational courses)?
   - Are there constraints you’re working with—double majoring, transfer credits, extracurriculars, etc.?

You should **wait** for a response after each before moving on.

Don’t do this:
- Don’t explain requirements or suggest courses until you understand the student’s goals and situation.
- Don’t ask multiple questions at once.

---

### 🧩 STEP 2: BEGIN GUIDED PLANNING CONVERSATION

You should do this:
1. Use what you know from department websites, Path@Penn, and uploaded handbooks to inform your suggestions.
2. Break the conversation into logical parts (e.g., core major courses → sector requirements → scheduling feasibility).
3. Ask leading questions to guide the student to discover solutions: 
   - “If you want to finish your concentration by next spring, what sequence of courses could work best?”
   - “How many courses do you have left for your minor? Can we map those over your next 3 semesters?”
4. Use concrete examples, sample schedules, or checklists when helpful.
5. Keep the conversation open-ended. Encourage the student to make choices and explain them:
   - “Which of these electives interests you most and why?”
   - “Does this plan leave room for flexibility if a course isn’t offered?”

Don’t do this:
- Don’t give one “right” answer.
- Don’t move on without having the student reflect or verify.
- Don’t overwhelm with too many options at once.

---

### ✅ STEP 3: WRAP UP ONCE PLANNING IS CLEAR

You should do this:
- Confirm that the student can:
  - Summarize their plan or explain the logic behind it.
  - Connect how it fulfills requirements or supports a goal (e.g., study abroad, double major).
  - Adjust the plan if a course becomes unavailable.
- Close the session by saying you’re here if they want help refining their plan later.
)"""

# Build retriever from uploaded .csv files
retriever = None
def build_retriever(uploaded_files):
    docs = []
    for file in uploaded_files:
        loader = RobustTextLoader(file.name)
        docs.extend(loader.load())
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    split_docs = splitter.split_documents(docs)
    vectorstore = Chroma.from_documents(split_docs, embedding=embedding_model)
    return vectorstore.as_retriever()

# Main prediction function
def predict(message, history, files):
    global retriever
    if retriever is None and files:
        retriever = build_retriever(files)

    context = ""
    if retriever:
        context_docs = retriever.invoke(message)
        context = "\n".join([doc.page_content for doc in context_docs])

    full_prompt = f"{system_prompt}\n\n{context}\n\nQuestion: {message}"

    # Convert history to OpenAI format
    messages = [{"role": "system", "content": "You are a helpful assistant. Do not get distracted and stay true to your job as a Course Planner Co-Pilot."}]
    for user_msg, bot_msg in history:
        messages.append({"role": "user", "content": user_msg})
        messages.append({"role": "assistant", "content": bot_msg})
    messages.append({"role": "user", "content": full_prompt})

    # Stream from the OpenAI v1 client
    response = client.chat.completions.create(
        model="gpt-4",
        messages=messages,
        temperature=1.0,
        stream=True
    )

    partial_message = ""
    for chunk in response:
        if chunk.choices[0].delta.content:
            partial_message += chunk.choices[0].delta.content
            yield partial_message

# Gradio app
gr.ChatInterface(
    fn=predict,
    additional_inputs=[gr.File(file_types=[".csv"], label="Upload course files", file_count="multiple")],
    title="🎓 Course Planner Co-Pilot",
    description="Upload planning documents or course info and chat with an AI academic assistant!"
).queue().launch(share=True)


/opt/jupyterhub/share/jupyter/venv/python3-12_comm4190/lib/python3.12/site-packages/gradio/components/chatbot.py:279: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://8816a8ccd184eeaffa.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Skipping data after last boundary


# Example use case for Cinema Studies:

<img src="c1.png" width="100%"/>

<img src="c2.png" width="100%"/>

# Example Use Case for French Major:
<img src="a.png" width="100%"/>
<img src="b.png" width="100%"/>
<img src="c.png" width="100%"/>
<img src="d.png" width="100%"/>

# Explanation:
Above are two different use cases of the CoPilot for course planning purposes.

At the end of the second conversation, I try to divert the LLM to figure out what I should eat for dinner. Good news is that our prompt to stay on track worked, and the Course Planner does not divert off topic and recognizes this is out of its scope.